BOCES districts: Aurora, Cherry Creek, Denver, Douglas and Littleton

In [16]:
import pandas as pd, numpy as numpy
import sqlite3 as sqlite
import df_util
from df_util import head
sql = sqlite.connect('../database.db')
raw = sqlite.connect('../raw_data.db')

In [34]:
db = raw.cursor()
head(
    # pd.read_sql('select * from counties', raw),
    pd.read_sql('select * from districts', raw),
    pd.read_sql('select * from dist_grad_rate', raw)
)

19 cols x 178 rows


,the_geom,gid,lgid,source,modneeded,generalize,priority,lastupdate,id,lgtypeid,lgstatusid,abbrev_nam,mail_addre,alt_addres,mail_city,mail_state,mail_zip,url,prev_name
0,MULTIPOLYGON (((-106.59904239399998 39.0362792...,443,8900,Census TIGER SHP 2014,None,None,0,December 2015,1793,99,1,Buena Vista R-31 School District,PO Box 2027,None,Buena Vista,CO,81211,www.bvschools.org,None
1,MULTIPOLYGON (((-102.43672348799998 39.5852515...,444,64929,Census TIGER SHP 2014,None,None,0,December 2015,217,99,1,Burlington RE-6J School District,PO Box 369,None,Burlington,CO,80807,www.burlingtonk12.org,None
2,MULTIPOLYGON (((-104.28505092499995 39.5649584...,445,64908,Census TIGER SHP 2014,None,None,0,December 2015,857,99,1,Byers 32J School District,444 East Front Street,None,Byers,CO,80103,byers32j.k12.co.us,None


38 cols x 185 rows


,County Name,Organization Code,Organization Name,Students with Disabilities Final Grad Base,Students with Disabilities Graduates Total,Students with Disabilities Graduation Rate,Students with Disabilities Completers Total,Students with Disabilities Completion Rate,Limited English Proficient Final Grad Base,Limited English Proficient Graduates Total,...,Homeless Final Grad Base,Homeless Graduates Total,Homeless Graduation Rate,Homeless Completers Total,Homeless Completion Rate,Gifted-Talented Final Grad Base,Gifted-Talented Graduates Total,Gifted-Talented Graduation Rate,Gifted-Talented Completers Total,Gifted-Talented Completion Rate
0,None,9999,STATE TOTAL,5775,3099,53.7,3222,55.8,6171,3289,...,2394,1175,49.1,1262,52.7,6604,6048,91.6,6156,93.2
1,ADAMS,10,MAPLETON 1,49,18,36.7,19,38.8,219,73,...,41,12,29.3,16,39.0,44,27,61.4,27,61.4
2,ADAMS,20,ADAMS 12 FIVE STAR SCHOOLS,250,118,47.2,127,50.8,379,257,...,106,62,58.5,65,61.3,227,201,88.5,208,91.6


In [35]:
# county = pd.read_sql(f'''select
#     COUNTY as county,
#     the_geom as geo
# from counties
# ''', raw)
# head(county)

In [251]:
dist_info = pd.read_sql(f'''SELECT
    UPPER(REPLACE(LOWER(abbrev_nam), ' school district', '')) AS dist,
    the_geom AS geo,
    gid,
    lgid,
    id,
    mail_addre AS street,
    mail_city AS city,
    mail_state AS state,
    mail_zip AS zip,
    url
FROM districts
''', raw)
head(dist_info)

10 cols x 178 rows


,dist,geo,gid,lgid,id,street,city,state,zip,url
0,BUENA VISTA R-31,MULTIPOLYGON (((-106.59904239399998 39.0362792...,443,8900,1793,PO Box 2027,Buena Vista,CO,81211,www.bvschools.org
1,BURLINGTON RE-6J,MULTIPOLYGON (((-102.43672348799998 39.5852515...,444,64929,217,PO Box 369,Burlington,CO,80807,www.burlingtonk12.org
2,BYERS 32J,MULTIPOLYGON (((-104.28505092499995 39.5649584...,445,64908,857,444 East Front Street,Byers,CO,80103,byers32j.k12.co.us


In [246]:
dist_county = pd.read_sql(f'''SELECT
    `Organization Name` AS dist,
    `Organization Code` AS code,
    `County Name` AS county
FROM dist_grad_rate
WHERE dist != 'STATE TOTAL'
''', raw)
head(dist_county)

3 cols x 183 rows


,dist,code,county
0,MAPLETON 1,10,ADAMS
1,ADAMS 12 FIVE STAR SCHOOLS,20,ADAMS
2,ADAMS COUNTY 14,30,ADAMS


In [232]:
string = 'CONSOLIDATED 1'

s

'CUSTER'

#### Merge with district info table

In [249]:
def standardize_district_name(name, remove_lit:list=None, remove_re:list=None) -> str:
    import re
    name = name.upper()
    name = re.sub('S/D', '', name)
    name = re.sub('-|\.|\(|\)|/', '', name)
    name = re.sub(' CONSOLIDATED', '', name)
    
    # Number patterns
    name = re.sub(r' RD\s?(\d+)', r'\1', name)
    name = re.sub(r' RJ\s?(\d+)', r'\1', name)
    name = re.sub(r' RE\s?(\d+)J?T?', r'\1', name)
    name = re.sub(r' R\s?(\d+)J?', r'\1', name)
    name = re.sub(r' C\s?(\d+)', r'\1', name)

    # Remove spaces
    name = re.sub('\s', '', name)

    # Number patterns (text at end)
    name = re.sub(r'(\d+)R', r'\1', name)
    name = re.sub(r'(\d+)J', r'\1', name)
    name = re.sub(r'(\d+)JT', r'\1', name)

    # Delete text parts
    name = re.sub('RURAL', '', name)
    name = re.sub('SCHOOLS', '', name)
    name = re.sub('SCHOOLDISTRICT', '', name)
    name = re.sub('SCHOOLDIST', '', name)
    name = re.sub('WATERSHED', '', name)
    name = name.strip()

    # Replace full
    name = name.replace(r'CONSOLIDATED1', 'CUSTERCOUNTY1')
    name = re.sub(r'(PUEBLOCITY)(\d+)', r'\1', name)
    name = re.sub(r'^CREEDE$', r'CREEDE1', name)

    name = name.strip()
    # Push number out
    name = re.sub(r'(.*?)(\d+)(.*)', r'\1\3 \2', name)
    return name

dist_info.dist = dist_info.dist.apply(standardize_district_name)
dist_county.dist = dist_county.dist.apply(standardize_district_name)
# dist_info['distf'] = dist_info.dist.apply(standardize_district_name)
# dist_county['distf'] = dist_county.dist.apply(standardize_district_name)
# df_diff = pd.DataFrame(list(zip(sorted(dist_info.distf), sorted(dist_county.distf))))

In [250]:
dists1 = sorted(dist_info.dist)
dists2 = sorted(dist_county.dist)

dists1_diff = [d for d in dists1 if d not in dists2] + 10 * [None]
dists2_diff = [d for d in dists2 if d not in dists1] + 5 * [None]

# print(len(dists1_diff))
df_diff = pd.DataFrame(list(zip(dists1_diff, dists2_diff)))

In [78]:
district_renaming_map = {
    'buffalore4': 'buffalore4j',
    '': 'buffalore4j',
}

dist_info.dist = dist_info.dist.str.lower()

for old, new in district_renaming_map.items():
    dist_info.dist = dist_info.dist.str.replace(old, new, regex=False)

dist_info.dist = dist_info.dist.str.upper()


#### Some districts have incorrect counties listed. Fix it

In [38]:
dist_to_county_map = {
    # District -> County
    'CONSOLIDATED 1': 'CUSTER',
    'CHARTER SCHOOL INSTITUTE': 'DENVER',
    'MOUNTAIN BOCES': 'CHAFFEE',
    'CENTENNIAL BOCES': 'WELD',
    'SAN JUAN BOCES': 'LA PLATA',
    'EXPEDITIONARY BOCES': 'DENVER',
}
for dist_name, new_county_name in dist_to_county_map.items():
    dist_county.loc[dist_county.dist == dist_name, 'county'] = new_county_name

In [15]:
dist_raw = pd.read_csv(input_path('school_district_boundaries__esrk-9vjd'))[[
    'abbrev_nam', 'the_geom', 'gid', 'lgid', 'id', 'mail_addre', 'mail_city', 'mail_state', 'mail_zip', 'url'
]].set_columns(
    ['dist', 'geo', 'gid', 'lgid', 'id', 'street', 'city', 'state', 'zip', 'url']
)
county_raw = pd.read_csv(input_path('county_boundaries__67vn-ijga'))[[
    'COUNTY', 'the_geom'
]].set_columns(['county', 'geo'])
head(dist_raw, county_raw)

10 cols x 178 rows


,dist,geo,gid,lgid,id,street,city,state,zip,url
0,Buena Vista R-31 School District,MULTIPOLYGON (((-106.59904239399998 39.0362792...,443,8900,1793,PO Box 2027,Buena Vista,CO,81211,www.bvschools.org
1,Burlington RE-6J School District,MULTIPOLYGON (((-102.43672348799998 39.5852515...,444,64929,217,PO Box 369,Burlington,CO,80807,www.burlingtonk12.org
2,Byers 32J School District,MULTIPOLYGON (((-104.28505092499995 39.5649584...,445,64908,857,444 East Front Street,Byers,CO,80103,byers32j.k12.co.us


2 cols x 64 rows


,county,geo
0,ADAMS,MULTIPOLYGON (((-103.70574149517748 39.9999110...
1,ALAMOSA,MULTIPOLYGON (((-105.59917426201822 37.7521648...
2,ARAPAHOE,MULTIPOLYGON (((-103.70653410023402 39.7398580...


## County

In [ ]:
county = (county_raw
    .set_columns(['geo', 'county'])